# 4 Adjust data for different models

In [0]:
from google.colab import drive
import pandas as pd
import numpy as np
drive.mount('/content/drive')

* Read the Dataset

In [0]:
train = pd.read_csv('/content/drive/My Drive/WORKS/data/Method2/trainLabeled.csv')
test = pd.read_csv('/content/drive/My Drive/WORKS/data/Method2/testLabeled.csv')
valid = pd.read_csv('/content/drive/My Drive/WORKS/data/Method2/validLabeled.csv')

* Format data for differet usages

In [0]:
def format(text):
# Format data for differet usages
  fact =[]
  fact1 = []
  label =[]
  label2 = []

  for i in range(len(text.Facts)):
    if text.Facts[i] == '。':
      fact1.append(text.Facts[i])
      label2.append(text.Label[i]+',')
      fact.append(fact1)
      label.append(label2)
      fact1 = []
      label2 = []
    else:
      fact1.append(text.Facts[i])
      label2.append(text.Label[i]+',')

  return fact,label

* Backup

In [0]:
import os

def storage(fileName,data):
# Store data
  with open(fileName,"w") as f:
    for i in data:
      f.writelines(str(i))
      f.writelines("\n")

* Run functions

In [0]:
train_x,train_y=format(train)
test_x,test_y=format(test)
valid_x,valid_y=format(valid)

In [0]:
path = '/content/drive/My Drive/WORKS/data/Method2/test/'

storage((path+'train_x.txt'),train_x)
storage((path+'train_y.txt'),train_y)
storage((path+'test_x.txt'),test_x)
storage((path+'test_y.txt'),test_y)
storage((path+'valid_x.txt'),valid_x)
storage((path+'valid_y.txt'),valid_y)

# 5 Train Models

In [0]:
pip uninstall tensorflow -y

In [0]:
pip install tensorflow-gpu==1.14.0 

In [0]:
pip install kashgari

## 5.1 Read the Dataset

In [0]:
import os

def readFile(fileName,type):
# Read labeled data
  text=[]

  with open(path+fileName+".txt", 'r', encoding='utf-8') as f:
    for line in f:
      d=[]
      sentence = line[:-1]
      if type == 1:
        for i in sentence:
          d+=i
      elif type == 0:
        d = sentence.split(',')
      text.append(d)
  return text

In [0]:
path = '/content/drive/My Drive/WORKS/data/Method2/test/3/'

train_x=readFile('train_x',1)
train_y=readFile('train_y',0)
valid_x=readFile('valid_x',1)
valid_y=readFile('valid_y',0)
test_x=readFile('test_x',1)
test_y=readFile('test_y',0)

## 5.2.1 Bi-LSTM_CRF Model

In [0]:
from kashgari.tasks.labeling import BiLSTM_CRF_Model
import kashgari

kashgari.config.use_cudnn_cell = True
LCModel = BiLSTM_CRF_Model()

In [0]:
historyBLSTM = LCModel.fit(train_x,train_y,
          valid_x,valid_y,
          batch_size=256,
          epochs=8)

LCModel.save('/content/drive/My Drive/WORKS/data/Method2/Out/BLSTMModel')

In [0]:
LCModel.evaluate(test_x,test_y)

## 5.2.2 CNN+Bi-LSTM_CRF Model



In [0]:
from kashgari.tasks.labeling import CNN_LSTM_Model
import kashgari

kashgari.config.use_cudnn_cell = True
CLCModel = CNN_LSTM_Model()
hyper = CLCModel.get_default_hyper_parameters()

In [0]:
hyper['layer_conv']['filters'] = 256
hyper['layer_conv']['kernel_size'] = 9

In [0]:
historyBLSTM = CLCModel.fit(train_x,train_y,
          valid_x,valid_y,
          batch_size=256,
          epochs=8)

CLCModel.save('/content/drive/My Drive/WORKS/data/Method2/Out/9-256CLCModel')

In [0]:
CLCModel.evaluate(test_x,test_y)

## 5.3 Word2vec_LSTM_CRF Model

* sgns.target.word-word.dynwin5.thr10.neg5.dim300.iter5.bz2
* sgns.target.word-character.char1-2.dynwin5.thr10.neg5.dim300.iter5.bz2
* sgns.baidubaike.bigram-char.bz2
* sgns.renmin.word.bz2

In [0]:
from kashgari.embeddings import WordEmbedding
from kashgari.tasks.labeling import BiLSTM_CRF_Model
import kashgari

kashgari.config.use_cudnn_cell = True

embedding = WordEmbedding('/content/drive/My Drive/WORKS/data/Pretrained_Model/sgns.target.word-word.dynwin5.thr10.neg5.dim300.iter5.bz2', 
                           task=kashgari.LABELING,
                          sequence_length=128)

In [0]:
w2vecModel = BiLSTM_CRF_Model(embedding)
hyper = w2vecModel.get_default_hyper_parameters()

In [0]:
historyWordvec = w2vecModel.fit(train_x,train_y,
          valid_x,valid_y,
          epochs=5,
          batch_size=256)

w2vecModel.save('/content/drive/My Drive/WORKS/data/Method2/Out/RenMinWord2vecBLSTMModel')

In [0]:
w2vecModel.evaluate(test_x,test_y)

## 5.4 Bert_LSTM_CRF Model

* chinese_rbt3_L-3_H-768_A-12
* chinese_rbtl3_L-3_H-1024_A-16
* chinese_L-12_H-768_A-12
* bert_zh_L-12_H-768_A-12

In [0]:
import kashgari
from kashgari.embeddings import BERTEmbedding

kashgari.config.use_cudnn_cell = True
bert_embed = BERTEmbedding('/content/drive/My Drive/WORKS/data/Pretrained_Model/chinese_rbt3_L-3_H-768_A-12',
                           task=kashgari.LABELING,
                           sequence_length=128)

* Divide train set to 100%, 67%, 50% to test

In [0]:
from sklearn.model_selection import train_test_split
# split data into train and test sets

seed = 42
test_size = 0.33
X_train,X_test,y_train,y_test = train_test_split(train_x,train_y,
test_size=test_size,random_state=seed)

In [0]:
from kashgari.tasks.labeling import BiLSTM_CRF_Model

bertModel = BiLSTM_CRF_Model(bert_embed)

historyBert = bertModel.fit(train_x,train_y,
          epochs=5,
          batch_size=256)

bertModel.save('/content/drive/My Drive/WORKS/data/Method2/Out/baserbt100BertBLSTMModel')

In [0]:
BertPath = '/content/drive/My Drive/WORKS/data/Method2/Out/baserbt50BertBLSTMModel'
trained_model = kashgari.utils.load_model(BertPath)

In [0]:
bertModel.evaluate(test_x,test_y)

## 5.3 GPT2 LSTM CRF Model

In [0]:
import kashgari
from kashgari.embeddings import GPT2Embedding

kashgari.config.use_cudnn_cell = True

gpt_embed = GPT2Embedding('/content/drive/My Drive/WORKS/data/Pretrained_Model/model.ckpt-100000.data-00000-of-00001',
                           task=kashgari.LABELING,
                           sequence_length=100)

In [0]:
from kashgari.tasks.labeling import BiLSTM_CRF_Model

model = BiLSTM_CRF_Model(gpt_embed)

historyGPT = model.fit(train_x,train_y,
          valid_x,valid_y,
          epochs=5,
          batch_size=256)

model.save('/content/drive/My Drive/WORKS/data/Method2/Out/GPT2BLSTMModel')

In [0]:
history = model.evaluate(test_x,test_y)